# Encounters between ants
Function to calculate the encounters between specific ants (e.g., focal and caregiver ants) from a ```.mymridon``` experiment file. <br><br>
There is probably an easier way to do this by querying individual frames directly.

In [ ]:
from datetime import datetime, timedelta  # For convenient handling of time and date

import numpy as np
import pandas as pd  # Used to create a dataframe, similar to the structure used in R
import py_fort_myrmidon as fm

### Function to output trajectories of all ants

In [ ]:
def trajectory_output_all(start_time, end_time, exp):
    """
    Function to extract daily trajectories, grouped by AntID. While it is setup to extract daily trajectories, it can work for any arbitrary time duration
    :param start_time: The start datetime object. this will be converted to a fort-myrmidon Time object
    :param end_time: The end datetime object. this will be converted to a fort-myrmidon Time object
    :param exp: The name of the experiment i.e., the myrmidon file
    :param matcher_query: The fm matcher corresponding to the focal IDs
    :return: Outputs a pandas dataframe containing AntID, Space, Time, X_coordinates and Y_coordinates of each ID averaged over 1 second from the X and Y coordinates. Averagingg is done to have a dataset which can be merged across IDs using at the resolution of 1s.
    """
    start = datetime.now()
    t_begin = fm.Time(start_time)
    t_stop = fm.Time(end_time)
    trajectory = fm.Query.ComputeAntTrajectories(
        experiment=exp,
        start=t_begin,
        end=t_stop,
        # matcher=matcher_query,
        maximumGap=fm.Duration.Parse("1000h"),
        reportProgress=False,
    )
    # Make a list of lists with trajectory values needed. Position is an array of 5 columns, so specific columns are called
    traj_list = [
        [
            trajectory.Ant,
            trajectory.Space,
            trajectory.Start.ToDateTime(),
            trajectory.Positions[:, 0],
            trajectory.Positions[:, 1],
            trajectory.Positions[:, 2],
        ]
        for trajectory in trajectory
    ]
    # Make the list into a dataframe
    traj_df = pd.DataFrame(
        traj_list,
        columns=["AntID", "Space", "StartTime", "Pos_time", "X_coor", "Y_coor"],
    )
    # Explode columns which are in the form of lists to expand the dataframe
    traj_df = traj_df.explode(column=["Pos_time", "X_coor", "Y_coor"])
    # Coerce coordinates to integer
    traj_df["X_coor"] = pd.to_numeric(traj_df["X_coor"], errors="coerce")
    traj_df["Y_coor"] = pd.to_numeric(traj_df["Y_coor"], errors="coerce")
    # Convert Pos_time to timedelta and obtain actual datetime for each trajectory entry
    traj_df["Pos_time"] = pd.to_numeric(traj_df["Pos_time"], errors="coerce")
    traj_df["Pos_time"] = pd.to_timedelta(
        traj_df["Pos_time"], unit="S", errors="coerce"
    )
    traj_df["Time"] = traj_df["StartTime"] + traj_df["Pos_time"]
    # Drop unwanted ccolumns
    traj_df = traj_df.drop(["StartTime", "Pos_time"], axis=1)
    # Reorder columns
    traj_df = traj_df[["AntID", "Space", "Time", "X_coor", "Y_coor"]]
    if traj_df.empty:  # If no trajectories are output
        # empty_row = pd.DataFrame([{'AntID': 'Unknown', 'Space':np.nan, 'Time':np.nan, 'X_coor':np.nan, 'Y_coor':np.nan}]) # Create empty row with unknown as antID
        # traj_df = pd.concat([empty_row]) # Add empty row to dataframe
        print("No trajectories found. Created empty dataframe")
        return traj_df  # Return empty dataframe
    # Obtain average X and Y coordinates per second
    # traj_df = (
    #     traj_df.groupby([pd.Grouper(key="Time", freq="1s"), "AntID", "Space"])
    #     .agg(X_mean=("X_coor", "mean"), Y_mean=("Y_coor", "mean"))
    #     .reset_index()
    # )
    end = datetime.now()
    # print("Trajectories output in", end-start)
    return traj_df

### Function to calculate duration of encounters
This function will calculate the duration of time an ant spends in the encounter zone after moving from the away zone to the encounter zone. This will require first identifying instances of an encounter, then calculating duration when the ant is within the encounter threshold

In [ ]:
# Function to run over each sequence of encounters to extract sub-sequences where individuals are within the encounter threshold (enc_dummy=1).
# The number and total duration of these sequences is output
def time_within_encounter_threshold(displacement_dataset, encounter_sequence):
    """Function to quantify sub-sequences when an individual is within the encounter threshold during an encounter.
    These sub-sequences have a enc_dummy value of 1. The function extracts these sub-sequences, obtains their start and end time points
    from the displacment dataset and outputs the number and total duration of all sub-sequences within an encounter sequence.
    Note that the encounter sequence should only contain values of 0.5 and 1, since an encounter by definition is within two instances of enc_dummy == 0

    Args:
        displacement_dataset (pandas.DataFrame): A pandas dataframe which contains at least the main index values and a column with Time
        encounter_sequence (numpy.array): A numpy array containing the sequence of enc_dummy values with 0.5 and 1 corresponding to when displacement is within away threshold and within encounter threshold

    Returns:
        total_enc (int): The number of sub-sequences which are within the encounter threshold
        total_enc (numpy.float64): The total duration of all sub-sequences which are within the encounter threshold
    """
    # Add 0.5 to end of the list, then find out indices where consecutive difference is not 0
    change_indices = np.where(
        np.diff(np.concatenate(([0.5], encounter_sequence, [0.5]))) != 0
    )[0]
    # Get start indices starting from the bginning and jumping by 2 upto the end
    start_indices = [encounter_sequence.index.values[x] for x in change_indices[::2]]
    # Get end indices starting from 1 going upto the end jumping by 2. Subtract 1 from all index values to account for the extra value added at the concatenation step
    end_indices = [encounter_sequence.index.values[x] for x in change_indices[1::2] - 1]
    # Get time of starting indices
    start_times = [displacement_dataset.loc[x, "Time"] for x in start_indices]
    # Get time of ending indices
    end_times = [displacement_dataset.loc[x, "Time"] for x in end_indices]
    # Get time duration within encounter thresholds by subtraction
    enc_times = np.subtract(end_times, start_times)
    enc_times_sec = [x.total_seconds() for x in enc_times]
    # Obtain total time and number of times ants are within encounter threshold consecutively
    total_enc_times = np.sum(enc_times_sec)
    total_enc = len(enc_times_sec)
    return total_enc, total_enc_times

In [ ]:
def encounter_duration(displacement_dataset, encounter_threshold, away_threshold):
    """Function to calculate duration of encounters between a focal ant and another individual.
    Encounters are defined as when an ant moves from beyond the away threshold to within the encounter threshold with respect to the focal ant.
    Two metrics of encounter duration are calculated - one based on the total time from when an ant moves within the encounter threshold to when it crosses the away threshold again
    and another based on only the total time spent within the encounter threshold during the encounter
    Outputs a dataframe with these durations calculated for each encounter between the 2 ants

    Args:
        displacement_dataset (pandas.DataFrame): A pandas dataframe containing at least Timestamps and displacement between 2 ants at each timestamp. It should also contain an index which is used to match sequence starts and stops to timestamps
        encounter_threshold (int): The value of displacement for the encounter threshold
        away_threshold (int): The value of displacement for the away threshold

    Returns:
        enc_df(pandas.DataFrame): A dataframe containing the number of the encounter, the start time, total duration,
    number of times within the encounter where the displacement between the two ants were within the encounter threshold and the total duration of these phases
    """
    # First interpolate (linearly) missing displacement values
    displacement_dataset["disp"].interpolate(
        method="linear", limit_direction="forward", inplace=True
    )
    # Create a new column based on converting the thresholds to dummy numbers. Values given are 1, if displacement < encounter threshold, 0.5, if encounter threshold < disp < away threshold and 0 if disp > away threshold.
    # Due to the linear interpolation there are no np.nans
    displacement_dataset.loc[:, ["enc_dummy"]] = pd.cut(
        displacement_dataset.disp,
        [0, encounter_threshold, away_threshold, np.inf],
        labels=[1, 0.5, 0],
    )
    # Convert datatype to float from category (due to pd.cut) for downstream functions
    displacement_dataset = displacement_dataset.astype({"enc_dummy": float})
    # Check if 1 is present at least once in the dataset. If not create a dataframe with 0 values to return
    if 1 not in displacement_dataset.enc_dummy.values:
        enc_df = pd.DataFrame(data=[[0, np.nan, 0.0, 0, 0.0]])
        enc_df.columns = [
            "enc_number",
            "enc_start_time",
            "enc_duration",
            "enc_sequences",
            "enc_sequences_duration",
        ]
        return enc_df
    # Identify index values where enc_dummy is 0 i.e., displacement is > away threshold
    away_indices = displacement_dataset[
        displacement_dataset["enc_dummy"] == 0
    ].index.values
    # Check if there is none or only 1 index with enc_dummy=0
    if away_indices.size < 2:
        enc_df = pd.DataFrame(data=[[0, np.nan, 0.0, 0, 0.0]])
        enc_df.columns = [
            "enc_number",
            "enc_start_time",
            "enc_duration",
            "enc_sequences",
            "enc_sequences_duration",
        ]
        return enc_df
    # Check if first value is not starting index and insert it if not
    if away_indices[0] != np.take(displacement_dataset.index.values, 0):
        away_indices = np.insert(
            away_indices, 0, np.take(displacement_dataset.index.values, 0)
        )  # Insert starting value of index range
    # Get pairs of consecutive indices
    away_indices_pair = list(zip(away_indices, away_indices[1:]))
    # Get values between the consecutive indices with value of 1
    seq_bw_away = [
        displacement_dataset.loc[x + 1 : y - 1, "enc_dummy"]
        for (x, y) in away_indices_pair
    ]
    # Remove list elements which have only 1 index value
    seq_bw_away_sub = [x for x in seq_bw_away if x.size > 1]
    # Subset list elements which have at least 1 value of 1
    # This list of lists will be used for all subsequent calculations
    enc_seq = [x for x in seq_bw_away_sub if np.in1d(1, x)]

    # Calculate overall encounter duration as time from when an individual crosses threshold (enc_dummy=1) to when it crosses away threshold again (enc_dummy=0)
    # Get index values where 1 is present for first time in each list within encounter sequences.
    # Then use this to obtain the start times of the encounters
    enc_start_times = [
        displacement_dataset.loc[x.index.values[np.where(x == 1)[0][0]], "Time"]
        for x in enc_seq
    ]
    # Get index values of last time point in each encounter.
    # Then use this to obtain the end times of the whole encounters
    enc_end_times = [
        displacement_dataset.loc[x.index.values[-1], "Time"] for x in enc_seq
    ]
    # Subtract end and start times element wise and convert to seconds
    enc_time = np.subtract(enc_end_times, enc_start_times)
    enc_time_sec = [x.total_seconds() for x in enc_time]

    # Calculate number of instances within each encounter where the individual is within encounter threshold for a sequential period of time and the total duration of these instances
    # This is based on extracting all consecutive sequences where enc_dummy=1 and calculating the start and end time of these sequences
    # The function `time_within_encounter_threshold` runs over each encounter sequence to extract the sub-sequences
    enc_sub_seq = [
        time_within_encounter_threshold(displacement_dataset, x) for x in enc_seq
    ]
    enc_sub_seq_num = [x for x, y in enc_sub_seq]
    enc_sub_seq_time = [y for x, y in enc_sub_seq]

    # Create a dataframe combining all the paeameters
    # First calculate number of encounters
    enc_num = np.arange(1, len(enc_seq) + 1)
    # Create a dataframe
    enc_df = pd.DataFrame(
        data=[enc_num, enc_start_times, enc_time_sec, enc_sub_seq_num, enc_sub_seq_time]
    )
    # transpose dataframe
    enc_df = enc_df.T
    # Add column names
    enc_df.columns = [
        "enc_number",
        "enc_start_time",
        "enc_duration",
        "enc_sequences",
        "enc_sequences_duration",
    ]
    enc_df = enc_df.astype(
        {
            "enc_number": int,
            "enc_duration": float,
            "enc_sequences": int,
            "enc_sequences_duration": float,
        }
    )
    return enc_df

### Function to calculate encounter durations between specific ants

This function combines obtaining all the trajectories from the `trajectory_output_all` function, then rearranges this to obtain the displacement between the focal ant and all other ants at each time point (in this every second, since the trajectories are summarised to the nearest second by averaging the X and Y coordinate for each second). This dataset is then grouped by focalID and antID and then the `encounter_duration` function is run over these groups.

In [ ]:
def focal_encounters(
    start_time, end_time, exp, focal_ID, exp_day, encounter_threshold, away_threshold
):
    """
    Function to obtain trajectories for focal and caregiver antIDs, merge by time and calculate displacement of each caregiver ID from the focal ID at every second
    :param start_time: Starting time to obtain trajectories from. Passed on to function trajectory_output
    :param end_time: Ending time to obtain trajectories from. Passed on to function trajectory_output
    :param exp: Location of myrmidon file
    :param focal_ID: Injured AntID
    :param exp_day: Day of the experiment. This is added to the dataframe for identification
    :param encounter_threshold: Threshold displacement to use as encounter
    :param away_threshold: Threshold displacement to use as the start/end of an encounter
    :return: Returns a datafarme containing the Time (in bins of 1s based on function trajectory_output), the focal and caregiver ID, the space in which the focal and caregiver ants are present, and the displacement between them (calculated as np.nan if they are in different spaces. In a CSV output this will be converted to a blank entry).
    """
    start = datetime.now()
    # # Focal Ant matcher
    # focal_matcher = fm.Matcher.AntID(focal_ID)
    # # Caregiver individual matchers
    # # others = [fm.Matcher.AntID(x) for x in other_IDs]
    # # Create single matcher object by unpacking the list within an Or Matcher
    # #others_matcher = fm.Matcher.Or(*others)
    # # Focal Ant trajectory
    # focal_traj = trajectory_output(start_time, end_time, exp, focal_matcher)
    # All ant trajectories
    other_traj = trajectory_output_all(start_time, end_time, exp)
    # Focal ant trajectory
    focal_traj = other_traj[other_traj["AntID"] == focal_ID]
    # Sort Time column for both dataframes
    other_traj = other_traj.sort_values("Time")
    focal_traj = focal_traj.sort_values("Time")

    # If focal trajectory is an empty dataframe, create a dataframe with na values for encounter parameters
    if focal_traj.empty:
        full_traj = other_traj.rename(columns={"Space": "Space_ant"})
        full_traj["focalID"] = focal_ID
        full_traj["Space_focal"] = full_traj["disp"] = (
            np.nan
        )  # Create columns with na values
        full_traj = full_traj[
            ["Time", "focalID", "AntID", "disp", "Space_focal", "Space_ant"]
        ]
        full_traj["exp_day"] = exp_day
        full_traj = full_traj[full_traj["focalID"] != full_traj["AntID"]].reset_index()
        # Group data frame, create columns with na and output encounter dataframe
        enc_df = (
            full_traj.groupby(["exp_day", "focalID", "AntID"])
            .apply(lambda x: pd.Series([np.nan] * 5))
            .reset_index()
            .rename(
                columns={
                    0: "enc_number",
                    1: "enc_start_time",
                    2: "enc_duration",
                    3: "enc_sequences",
                    4: "enc_sequences_duration",
                }
            )
        )
        print(
            f"{'Focal ID trajectory is empty for list item '}{exp_day}{' .Returning dataframe with no displacement and encounters calculated'}"
        )
        return enc_df
    # If trajectory of all other individuals is an empty dataframe, create a dataframe with na values for encounter parameters
    if other_traj.empty:
        full_traj = focal_traj.rename(
            columns={"AntID": "focalID", "Space": "Space_focal"}
        )
        full_traj["AntID"] = full_traj["Space_ant"] = full_traj["disp"] = (
            np.nan
        )  # Create columns with na values
        full_traj = full_traj[
            ["Time", "focalID", "AntID", "disp", "Space_focal", "Space_ant"]
        ]
        full_traj["exp_day"] = exp_day
        # Group data frame, create columns with na and output encounter dataframe
        enc_df = (
            full_traj.groupby(["exp_day", "focalID", "AntID"])
            .apply(lambda x: pd.Series([np.nan] * 5))
            .reset_index()
            .rename(
                columns={
                    0: "enc_number",
                    1: "enc_start_time",
                    2: "enc_duration",
                    3: "enc_sequences",
                    4: "enc_sequences_duration",
                }
            )
        )
        print(
            f"{'Caregiver ID trajectories are empty for list item '}{exp_day}{' .Returning dataframe with no displacement and encounters calculated'}"
        )
        return enc_df

    # Merge focal and caregiver trajectories on Time column using merge_asof to match nearest time values
    full_traj = pd.merge_asof(
        other_traj,
        focal_traj,
        on="Time",
        suffixes=("_ant", "_focal"),
        direction="nearest",
        tolerance=pd.Timedelta("1s"),
    )
    # Obtain X coordinate and Y coordinate difference between Focal and Caregivers, for each row
    full_traj["X_diff"] = full_traj["X_coor_focal"] - full_traj["X_coor_ant"]
    full_traj["Y_diff"] = full_traj["Y_coor_focal"] - full_traj["Y_coor_ant"]
    # Obtain displacement
    full_traj["disp"] = np.linalg.norm(
        full_traj[["X_diff", "Y_diff"]].to_numpy(), axis=1
    )
    # Rename columns
    full_traj = full_traj.rename(
        columns={"AntID_focal": "focalID", "AntID_ant": "AntID"}
    )
    # Subset specific columns
    full_traj = full_traj[
        ["Time", "focalID", "AntID", "disp", "Space_focal", "Space_ant"]
    ]
    # Add experimental day
    full_traj["exp_day"] = exp_day
    # Remove instances where the focal ant's displacement is calculated wrt itself.
    full_traj = full_traj[full_traj["focalID"] != full_traj["AntID"]].reset_index(
        drop=True
    )
    # Replace with arbitrary high value of displacemeent if focal ant and caregiver are in different spaces. Use notnull to filter out instances where focal or caregiver space is not known. The higgh value will ensure that this case is always considered as > away_threshold in count_encounters function
    full_traj.loc[
        (
            (full_traj.Space_focal.notnull())
            & (full_traj.Space_ant.notnull())
            & (full_traj.Space_focal != full_traj.Space_ant)
        ),
        "disp",
    ] = 100000
    # Apply encounter_duration function over grouped dataframe, reset index and rename columns
    enc_df = (
        full_traj.groupby(["exp_day", "focalID", "AntID"])
        .apply(lambda x: encounter_duration(x, encounter_threshold, away_threshold))
        .reset_index()
        .drop("level_3", axis=1)
    )
    end = datetime.now()
    print(
        f"{'Encounters for experimental day '}{exp_day}{' calculated in '}{end - start}"
    )
    return enc_df

### Function to run encounter calculations over multiple phases of one experiment

In [ ]:
def calculate_encounters_cluster(exp_list, control_list, pre_list, post_list, colonyID):
    """
    Helper function to run focal_caregivers_disp_exp and count_encounters over multiple lists associated with different phases for one colony
    :param exp_list: List of start_time, end_time, exp, focalID, careggiverIDs and exp_day for Experimental phase
    :param control_list: List for control phase
    :param pre_list: List for pre-experimental phase
    :param post_list: List for post-experimental phase
    :param colonyID: colonyID corresponding to the myrmidon file
    :return: Dataframe combining the encounters for each of the 4 phases for one colony
    """
    print("Experimental phase")
    exp_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in exp_list
    ]
    exp_enc = pd.concat(exp_enc)
    exp_enc["phase"] = "Exp"
    print("Control phase")
    control_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in control_list
    ]
    control_enc = pd.concat(control_enc)
    control_enc["phase"] = "Control"
    print("Pre-Experimental phase")
    pre_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in pre_list
    ]
    pre_enc = pd.concat(pre_enc)
    pre_enc["phase"] = "Pre"
    print("Post-Experimental phase")
    post_enc = [
        focal_encounters(
            start, end, exp, focalID, exp_day, encounter_threshold, away_threshold
        )
        for start, end, exp, focalID, exp_day, encounter_threshold, away_threshold in post_list
    ]
    post_enc = pd.concat(post_enc)
    post_enc["phase"] = "Post"
    # Combine encounter dataframes
    enc_list = [exp_enc, control_enc, pre_enc, post_enc]
    enc = pd.concat(enc_list)
    # Add colonyID
    enc["colony"] = colonyID
    # Sort by values
    enc = enc.sort_values(
        by=["colony", "phase", "exp_day", "focalID", "AntID"]
    ).reset_index(drop=True)
    enc = enc[
        [
            "colony",
            "phase",
            "exp_day",
            "focalID",
            "AntID",
            "enc_number",
            "enc_start_time",
            "enc_duration",
            "enc_sequences",
            "enc_sequences_duration",
        ]
    ]
    return enc

# Injury Experiments

In [ ]:
# Thresholds
encounter_threshold = 300  # threshold for counting as an encounter
away_threshold = 1000  # Threshold for counting as the end of an encounter

## Colony Cfel 42

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
# Create list of focal ants
focal = [106, 63, 23, 53, 19]
# Experimental phase
day_starts_exp = [
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
]
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]
exp_days = [1, 2, 3, 4, 5]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control Phase
day_starts_control = list(
    np.repeat(datetime(2022, 5, 1, 15, 54).astimezone(tz=None), 5)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre experimental phase
day_starts_pre = [
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
]
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase
day_starts_post = [
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 7, 9, 0).astimezone(tz=None),
]
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel42_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel42"
)

In [ ]:
cfel42_enc.to_csv("Cfel42_AllAnts_Focal_Encounters.csv", index=False)

## Colony Cfel 1

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
# Focal Ants
focal = [87, 37, 58, 38, 3]
exp_days = [1, 2, 3, 4, 5]
# Experimental Phase list
day_starts_exp = [
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 14, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
]
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control phase list
day_starts_control = list(
    np.repeat(datetime(2022, 6, 4, 14, 48).astimezone(tz=None), 5)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre Experimental phase list
day_starts_pre = [
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
]
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase list
day_starts_post = [
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
]
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel1_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel1"
)

In [ ]:
cfel1_enc.to_csv("Cfel1_AllAnts_Focal_Encounters.csv", index=False)

## Colony Cfel 54

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
# Focal Ants
focal = [108, 114, 62, 12]
exp_days = [1, 2, 3, 4]
# Experimental Phase list
day_starts_exp = [
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
]
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control phase list
day_starts_control = list(
    np.repeat(datetime(2022, 6, 19, 14, 25).astimezone(tz=None), 4)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre Experimental phase list
day_starts_pre = [
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
]
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase list
day_starts_post = [
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
]
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel54_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel54"
)

In [ ]:
cfel54_enc.to_csv("Cfel54_AllAnts_Focal_Encounters.csv", index=False)

# Infection Experiments

## Colony Cfel 13

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-3/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
# Focal Ants
focal = [9, 82, 40, 7, 55, 80, 26, 22, 27, 98]
exp_days = [1, 2, 3, 4, 5] * 2
# Experimental Phase list
day_starts_exp = [
    datetime(2023, 4, 24, 15, 29).astimezone(tz=None),
    datetime(2023, 4, 25, 14, 19).astimezone(tz=None),
    datetime(2023, 4, 26, 15, 3).astimezone(tz=None),
    datetime(2023, 4, 27, 16, 43).astimezone(tz=None),
    datetime(2023, 4, 28, 14, 27).astimezone(tz=None),
] * 2
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control phase list
day_starts_control = list(
    np.repeat(datetime(2023, 4, 23, 15, 5).astimezone(tz=None), 10)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre Experimental phase list
day_starts_pre = [
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 26, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
] * 2
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase list
day_starts_post = [
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 26, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 29, 8, 0).astimezone(tz=None),
] * 2
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel13_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel13"
)

In [ ]:
cfel13_enc.to_csv("Cfel13_AllAnts_Focal_Encounters.csv", index=False)

## Colony Cfel 55

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-3/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
# Focal Ants
focal = [30, 36, 44, 53, 55, 72, 15, 57, 67, 81]
exp_days = [1, 2, 3, 4, 5] * 2
# Experimental Phase list
day_starts_exp = [
    datetime(2023, 4, 20, 15, 45).astimezone(tz=None),
    datetime(2023, 4, 21, 14, 48).astimezone(tz=None),
    datetime(2023, 4, 22, 14, 17).astimezone(tz=None),
    datetime(2023, 4, 23, 14, 0).astimezone(tz=None),
    datetime(2023, 4, 24, 14, 54).astimezone(tz=None),
] * 2
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control phase list
day_starts_control = list(
    np.repeat(datetime(2023, 4, 18, 14, 40).astimezone(tz=None), 10)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre Experimental phase list
day_starts_pre = [
    datetime(2023, 4, 20, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 21, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 22, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 23, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
] * 2
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase list
day_starts_post = [
    datetime(2023, 4, 21, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 22, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 23, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
] * 2
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel55_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel55"
)

In [ ]:
cfel55_enc.to_csv("Cfel55_AllAnts_Focal_Encounters.csv", index=False)

## Colony Cfel 64

In [ ]:
f_myrmidon = "/media/ebiag/Ebi-1/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)
# Focal Ants
focal = [6, 104, 115, 78, 59, 32, 86, 1, 38, 3]
exp_days = [1, 2, 3, 4, 5] * 2
# Experimental Phase list
day_starts_exp = [
    datetime(2023, 6, 1, 15, 51).astimezone(tz=None),
    datetime(2023, 6, 2, 14, 44).astimezone(tz=None),
    datetime(2023, 6, 3, 14, 50).astimezone(tz=None),
    datetime(2023, 6, 4, 14, 43).astimezone(tz=None),
    datetime(2023, 6, 5, 14, 52).astimezone(tz=None),
] * 2
day_ends_exp = [day_time + timedelta(hours=6) for day_time in day_starts_exp]

disp_list_exp = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_exp, day_ends_exp, focal, exp_days)
]

# Control phase list
day_starts_control = list(
    np.repeat(datetime(2023, 5, 31, 15, 5).astimezone(tz=None), 10)
)
day_ends_control = [day_time + timedelta(hours=6) for day_time in day_starts_control]

disp_list_control = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_control, day_ends_control, focal, exp_days
    )
]

# Pre Experimental phase list
day_starts_pre = [
    datetime(2023, 6, 1, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
] * 2
day_ends_pre = [day_time + timedelta(hours=6) for day_time in day_starts_pre]

disp_list_pre = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(day_starts_pre, day_ends_pre, focal, exp_days)
]

# Post experimental phase list
day_starts_post = [
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 6, 8, 0).astimezone(tz=None),
] * 2
day_ends_post = [day_time + timedelta(hours=6) for day_time in day_starts_post]

disp_list_post = [
    (start, end, exp, focal, exp_day, encounter_threshold, away_threshold)
    for start, end, focal, exp_day in zip(
        day_starts_post, day_ends_post, focal, exp_days
    )
]

In [ ]:
cfel64_enc = calculate_encounters_cluster(
    disp_list_exp, disp_list_control, disp_list_pre, disp_list_post, "Cfel64"
)

In [ ]:
cfel64_enc.to_csv("Cfel64_AllAnts_Focal_Encounters.csv", index=False)